### I WAS NOT ABLE TO RUN THIS NOTEBOOK ON MY COMPUTER. I HAD EVERYTHING INSTALLED CORRECTLY BUT IT WAS TAKING 5 MINUTES TO CLASSIFY 10 UTTERANCES. NEEDLESS TO SAY, IT WAS GOING TO BE IMPOSSIBILE FOR ALL 190 UTTERANCES. 
I RAN THIS NOTEBOOK ON ANOTHER COMPUTER, TO WHICH I DID NOT HAVE UNLIMITED ACCESS. THIS IS ALL I COULD MANAGE TO DO.

I WILL STILL ATTACH TO DELIVERABLES MY NOTEBOOK WHERE I TRIED TO RUN THE MODEL MULTIPLE TIMES.

In [20]:
from langchain_ollama import ChatOllama
from datetime import datetime

In [21]:
annotation_act_labels = [
    "QUESTION",
    "ANSWER",
    "ACKNOWLEDGEMENT", "BACKCHANNEL",
    "DIRECTIVE", "REQUEST",
    "REPAIR", "CLARIFICATION",
    "EXPRESSIVE", "EMOTIVE",
    "APOLOGY",
    "GREETING",
    "GOODBYE", "CLOSING",
    "OTHER"
]

examples = [
    {'INPUT': 'we just ↑remi[NISCE]', 'OUTPUT': 'STATEMENT'},
    {'INPUT': '((laughs))', 'OUTPUT': 'OTHER'},
    {'INPUT': '[that- that is (.)] tr[ue]', 'OUTPUT': 'ACKNOWLEDGMENT'},
    {'INPUT': '°yeah°', 'OUTPUT': 'BACKCHANNEL'},
    {'INPUT': '°that was a good one°', 'OUTPUT': 'EXPRESSIVE'},
]

## Prompt try 1

I started out with Piek's code (HLT - 2025). I simply adjusted labels and instructions according to this task.

In [22]:
class LLMAnnotator:
    
    def __init__(self, model="qwen3:1.7b", labels=annotation_act_labels, examples=[], max_context=2):
        self._client = ChatOllama(
            model=model,
            think=False,
            temperature=0.2
        )
        self._max_context = max_context
        self._history = []
        self._instruct = []
        self.create_label_instruct(labels, examples)

    def create_label_instruct(self, labels=[], examples=[]):
        self._instruct = [{"role": "system", "content": "You are an expert annotator trained in Conversation Analysis and Dialogue Act tagging."}]
        self._instruct.append({"role": "system", "content": "You must assign exactly ONE dialogue act label to EACH utterance."})
        self._instruct.append({"role": "system", "content": 'Utterances use JEFFERSON TRANSCRIPTION. Use them as cues for emotion, repair, overlap, etc., but do NOT treat them as words.'})
        self._instruct.append({"role": "system", "content": "You must classify only the last utterance, using previous utterances only as context."})
        self._instruct.append({"role": "system", "content": "Output exactly one label in JSON format with the field 'label'. Do not output anything else."})
        self._instruct.append({"role": "system", "content": f"Only use one of the following labels: {labels}"})
        self._instruct.append({"role": "system", "content": "Use preceding utterances as context."})
        self._instruct.append({"role": "system", "content": "Output the most appropriate label in JSON format."})
        self._instruct.append({"role": "system", "content": "Do not output anything else."})
        if examples:
            self._instruct.append({"role": "system", "content": "Here are a few examples:"})
            for example in examples:
                self._instruct.append({"role": "user", "content": example["INPUT"]})
                self._instruct.append({"role": "system", "content": example["OUTPUT"]})

    def annotate_conversation(self, input=[]):
        annotations = []
        for utterance in input:
            annotation = self.annotate(utterance)
            annotations.append(annotation)
        return annotations

    def annotate(self, utterance):
        self._history.append({"role": "user", "content": f"INPUT: {utterance}"})

        if len(self._history) > self._max_context:
            self._history = self._history[1:]

        prompt = self._instruct + self._history
        response = self._client.invoke(prompt)

        # Prendi solo il contenuto testuale
        answer = response.content

        # Rimuovi eventuale <think> tag
        end_of_think = answer.find("</think>")
        if end_of_think > 0:
            answer = answer[end_of_think+8:]

        answer = answer.strip()  # pulito da spazi a inizio/fine

        return {"INPUT": utterance, "OUTPUT": answer}

In [15]:
with open('solo_frasi.txt', 'r') as infilez:
    all_file = infilez.read()
all_file = all_file.split('\n')
len(all_file)

196

In [17]:
llm_annotator  = LLMAnnotator(labels=annotation_act_labels, examples=examples, max_context=2)
annotations = llm_annotator.annotate_conversation(all_file[:10])
for annotation in annotations:
    print(annotation)

{'INPUT': 'we just â†‘remi[NISCE]', 'OUTPUT': '{"label": "EXPRESSIVE"}'}
{'INPUT': '[remi]nisce', 'OUTPUT': '{"label": "EXPRESSIVE"}'}
{'INPUT': '[eh]', 'OUTPUT': '{"label": "EXPRESSIVE"}'}
{'INPUT': '(a)[bout] a shared memory', 'OUTPUT': '{"label": "EXPRESSIVE"}'}
{'INPUT': "'kay", 'OUTPUT': '{"label": "EXPRESSIVE"}'}
{'INPUT': 'm::h hhh ((tsk))', 'OUTPUT': '{"label": "BACKCHANNEL"}'}
{'INPUT': 'which memory would you like', 'OUTPUT': '{"label": "QUESTION"}'}
{'INPUT': "but i'm tryin' to think which memory is (0.6) within five minutes because they're all short (.) cute moments", 'OUTPUT': '{"label": "QUESTION"}'}
{'INPUT': 'mh=yea:h', 'OUTPUT': '{"label": "CLARIFICATION"}'}
{'INPUT': "Â°we- we'll probably spend more than five minutes finding a memoryÂ°", 'OUTPUT': '{"label": "DIRECTIVE"}'}


## Prompt try 2

self_history edit > a little better but first labels are still non sense

In [23]:
class LLMAnnotator:
    
    def __init__(self, model="qwen3:1.7b", labels=annotation_act_labels, examples=[], max_context=2):
        self._client = ChatOllama(
            model=model,
            think=False,
            temperature=0.2
        )
        self._max_context = max_context
        self._history = []
        self._instruct = []
        self.create_label_instruct(labels, examples)

    def create_label_instruct(self, labels=[], examples=[]):
        self._instruct = [{"role": "system", "content": "You are an expert annotator trained in Conversation Analysis and Dialogue Act tagging."}]
        self._instruct.append({"role": "system", "content": "You must assign exactly ONE dialogue act label to EACH utterance."})
        self._instruct.append({"role": "system", "content": 'Utterances use JEFFERSON TRANSCRIPTION. Use them as cues for emotion, repair, overlap, etc., but do NOT treat them as words.'})
        self._instruct.append({"role": "system", "content": "You must classify only the last utterance, using previous utterances only as context."})
        self._instruct.append({"role": "system", "content": "Output exactly one label in JSON format with the field 'label'. Do not output anything else."})
        self._instruct.append({"role": "system", "content": f"Only use one of the following labels: {labels}"})
        self._instruct.append({"role": "system", "content": "Use preceding utterances as context."})
        self._instruct.append({"role": "system", "content": "Output the most appropriate label in JSON format."})
        self._instruct.append({"role": "system", "content": "Do not output anything else."})
        if examples:
            self._instruct.append({"role": "system", "content": "Here are a few examples:"})
            for example in examples:
                self._instruct.append({"role": "user", "content": example["INPUT"]})
                self._instruct.append({"role": "system", "content": example["OUTPUT"]})

    def annotate_conversation(self, input=[]):
        annotations = []
        for utterance in input:
            annotation = self.annotate(utterance)
            annotations.append(annotation)
        return annotations

    def annotate(self, utterance):
        self._history.append({"role": "user", "content": utterance})

        if len(self._history) > self._max_context:
            self._history = self._history[1:]

        prompt = self._instruct + self._history
        response = self._client.invoke(prompt)

        # Prendi solo il contenuto testuale
        answer = response.content

        # Rimuovi eventuale <think> tag
        end_of_think = answer.find("</think>")
        if end_of_think > 0:
            answer = answer[end_of_think+8:]

        answer = answer.strip()  # pulito da spazi a inizio/fine

        return {"INPUT": utterance, "OUTPUT": answer}

In [19]:
llm_annotator  = LLMAnnotator(labels=annotation_act_labels, examples=examples, max_context=2)
annotations = llm_annotator.annotate_conversation(all_file[:10])
for annotation in annotations:
    print(annotation)

{'INPUT': 'we just â†‘remi[NISCE]', 'OUTPUT': '{"label": "EXPRESSIVE"}'}
{'INPUT': '[remi]nisce', 'OUTPUT': '{"label": "EXPRESSIVE"}'}
{'INPUT': '[eh]', 'OUTPUT': '{"label": "ACKNOWLEDGEMENT"}'}
{'INPUT': '(a)[bout] a shared memory', 'OUTPUT': '{"label": "EMOTIVE"}'}
{'INPUT': "'kay", 'OUTPUT': '{"label": "REPAIR"}'}
{'INPUT': 'm::h hhh ((tsk))', 'OUTPUT': '{\n  "label": "ACKNOWLEDGEMENT"\n}'}
{'INPUT': 'which memory would you like', 'OUTPUT': '{"label": "QUESTION"}'}
{'INPUT': "but i'm tryin' to think which memory is (0.6) within five minutes because they're all short (.) cute moments", 'OUTPUT': '{"label": "CLARIFICATION"}'}
{'INPUT': 'mh=yea:h', 'OUTPUT': '{"label": "CLARIFICATION"}'}
{'INPUT': "Â°we- we'll probably spend more than five minutes finding a memoryÂ°", 'OUTPUT': '{"label": "DIRECTIVE"}'}


## Prompt try 3

In [ ]:
class LLMAnnotator:
    
    def __init__(self, model="qwen3:1.7b", labels=annotation_act_labels, examples=[], max_context=2):
        self._client = ChatOllama(
            model=model,
            think=False,
            temperature=0.2
        )
        self._max_context = max_context
        self._history = []
        self._instruct = []
        self.create_label_instruct(labels, examples)

    def create_label_instruct(self, labels=[], examples=[]):
        self._instruct = [{"role": "system", "content": "You are an expert annotator trained in Conversation Analysis and Dialogue Act tagging."}]
        self._instruct.append({"role": "system", "content": "You must assign exactly ONE dialogue act label to EACH utterance."})
        self._instruct.append({"role": "system", "content": 'Utterances use JEFFERSON TRANSCRIPTION. Use them as cues for emotion, repair, overlap, etc., but do NOT treat them as words.'})
        self._instruct.append({"role": "system", "content": "You must classify only the last utterance, using previous utterances only as context."})
        self._instruct.append({"role": "system", "content": "Output exactly one label in JSON format with the field 'label'. Do not output anything else."})
        self._instruct.append({"role": "system", "content": f"Only use one of the following labels: {labels}"})
        self._instruct.append({"role": "system", "content": "Use preceding utterances as context."})
        self._instruct.append({"role": "system", "content": "Output the most appropriate label in JSON format."})
        self._instruct.append({"role": "system", "content": "Do not output anything else."})
        if examples:
            self._instruct.append({"role": "system", "content": "Here are a few examples:"})
            for example in examples:
                self._instruct.append({"role": "user", "content": example["INPUT"]})
                self._instruct.append({"role": "system", "content": example["OUTPUT"]})

    def annotate_conversation(self, input=[]):
        annotations = []
        for utterance in input:
            annotation = self.annotate(utterance)
            annotations.append(annotation)
        return annotations

    def annotate(self, utterance):
        self._history.append({"role": "user", "content": utterance})

        if len(self._history) > self._max_context:
            self._history = self._history[1:]

        prompt = self._instruct + self._history
        response = self._client.invoke(prompt)

        # Prendi solo il contenuto testuale
        answer = response.content

        # Rimuovi eventuale <think> tag
        end_of_think = answer.find("</think>")
        if end_of_think > 0:
            answer = answer[end_of_think+8:]

        answer = answer.strip()  # pulito da spazi a inizio/fine

        return {"INPUT": utterance, "OUTPUT": answer}